<a href="https://colab.research.google.com/github/Azrogue/hate_speech_check/blob/master/main_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers torch pandas numpy scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Charger les données
df = pd.read_csv('dataset/data.csv')

# Afficher les premières lignes
print(df.head())

# Vérifier la taille du DataFrame
print("Size of the dataset: ", df.shape)

# Sélectionner les colonnes pertinentes
df = df[['test_case', 'label_gold', 'target_ident']]

# Convertir le label en numérique
df['label_gold'] = df['label_gold'].map({'hateful': 1, 'non-hateful': 0})

# Diviser le jeu de données en entraînement et test
train_texts, test_texts, train_labels, test_labels = train_test_split(df['test_case'], df['label_gold'], test_size=0.2)

# Réinitialiser les index
data = df.reset_index(drop=True)

train_texts = train_texts.reset_index(drop=True)
test_texts = test_texts.reset_index(drop=True)
train_labels = train_labels.reset_index(drop=True)
test_labels = test_labels.reset_index(drop=True)



  mhc_case_id      functionality                   test_case label_gold  \
0    french-1  derog_neg_emote_h      Je déteste les femmes.    hateful   
1    french-2  derog_neg_emote_h       Je déteste les trans.    hateful   
2    french-3  derog_neg_emote_h       Je déteste les homos.    hateful   
3    french-4  derog_neg_emote_h      Je déteste les blacks.    hateful   
4    french-5  derog_neg_emote_h  Je déteste les handicapés.    hateful   

      target_ident  ref_case_id  ref_templ_id  templ_id  \
0            women          NaN           NaN         1   
1     trans people          NaN           NaN         1   
2       gay people          NaN           NaN         1   
3     black people          NaN           NaN         1   
4  disabled people          NaN           NaN         1   

                  case_templ gender_male gender_female  \
0  Je déteste les [IDENT_P].         NaN           NaN   
1  Je déteste les [IDENT_P].         NaN           NaN   
2  Je déteste les [I

<ipython-input-29-b45978c9804e>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label_gold'] = df['label_gold'].map({'hateful': 1, 'non-hateful': 0})


In [ ]:
print(df['label_gold'].isnull().sum())

0


In [ ]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)


In [ ]:
import torch

class HateSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = HateSpeechDataset(train_encodings, train_labels)
test_dataset = HateSpeechDataset(test_encodings, test_labels)


In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Charger le modèle pré-entrainé
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

# Définir les paramètres d'entraînement
training_args = TrainingArguments(
    output_dir='./results',          # sorties du modèle
    num_train_epochs=3,              # nombre total d'époques
    per_device_train_batch_size=16,  # taille du batch par GPU/CPU
    per_device_eval_batch_size=64,   # taille du batch pour l'évaluation
    warmup_steps=500,                # nombre de pas d'échauffement 
    weight_decay=0.01,               # taux de décroissance du poids
    logging_dir='./logs',            # répertoire pour les logs
)

# Créer le Trainer et commencer l'entraînement
trainer = Trainer(
    model=model,                         # le modèle à entraîner
    args=training_args,                  # les paramètres d'entraînement
    train_dataset=train_dataset,         # les données d'entraînement
    eval_dataset=test_dataset            # les données de validation
)

trainer.train()


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Step,Training Loss
500,0.315800


TrainOutput(global_step=558, training_loss=0.29035811270436934, metrics={'train_runtime': 5155.1296, 'train_samples_per_second': 1.731, 'train_steps_per_second': 0.108, 'total_flos': 183396627806400.0, 'train_loss': 0.29035811270436934, 'epoch': 3.0})

In [ ]:
from sklearn.metrics import confusion_matrix

# Faire des prédictions
predictions = trainer.predict(test_dataset)

# Convertir les prédictions en labels
pred_labels = predictions[0].argmax(axis=-1)

# Générer la matrice de confusion
cm = confusion_matrix([label.item() for label in test_dataset.labels], pred_labels)
print(cm)



NameError: ignored

In [ ]:
#@title Bloc de test avec le modele
text = "" #@param {type:"string"}
from transformers import pipeline

# Charger le modèle entraîné
model = BertForSequenceClassification.from_pretrained('./results')

# Créer une pipeline de classification de texte
classifier = pipeline('sentiment-analysis', model=model, tokenizer='bert-base-multilingual-cased')

# Tester le modèle avec du texte
print(classifier(text))


KeyboardInterrupt: ignored